In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
import tensorflow as tf
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


CPU times: user 10.5 s, sys: 1.96 s, total: 12.5 s
Wall time: 16.3 s


In [ ]:
import pandas as pd

train_df = pd.read_csv('/content/Fake_train.csv', encoding='utf-8')
test_df = pd.read_csv('/content/Fake_test_without_labels.csv', encoding='utf-8')
df2 = pd.read_csv('/content/Fake_dev.csv', encoding='utf-8')


In [ ]:
import pandas as pd

# Load the dataset from the CSV file
df = pd.read_csv('/content/Fake_train.csv')

# Displaying DataFrame in Colab
df


,text,label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake
1,Masha Allah,Fake
2,"അന്വേഷണം കഴിയുമ്പോൾ,. C. A. A. യ്ക്ക് എതിരായ ക...",Fake
3,Illathentha avaru purath vidayittalland verenth,Fake
4,Barana pakshathin matoru niyamam.nalla moyanth...,original
...,...,...
3252,വീണാ ജോർജ് എന്ന ഒരു ആരോഗ്യ മന്ത്രി വന്നതിനു ശേ...,original
3253,ഇതാണ് പിണുവാതിര😂😂😂,original
3254,കേസ് എടുക്കണം 💯👍,original
3255,എല്ലാത്തിനേം 501 സോപ്പിട്ട് കളിപ്പിച്ച് വിടണം,original


In [ ]:
# Preprocess data
text_column = 'text'
label_column = 'label'
train_df[text_column] = train_df[text_column].fillna('')
test_df[text_column] = test_df[text_column].fillna('')

In [ ]:
# Tokenization
max_vocab_size = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df[text_column])

X_train = tokenizer.texts_to_sequences(train_df[text_column])
X_test = tokenizer.texts_to_sequences(test_df[text_column])
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')

In [ ]:
X_train

array([[  38,  328,  111, ...,    0,    0,    0],
       [4551, 1629,    0, ...,    0,    0,    0],
       [4552, 1229, 1630, ...,    0,    0,    0],
       ...,
       [ 184, 4430,    1, ...,    0,    0,    0],
       [   1,  891,    1, ...,    0,    0,    0],
       [ 739,  127,  739, ...,    0,    0,    0]], dtype=int32)

In [ ]:
# Label Encoding
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df[label_column])
print(train_df['label'].value_counts())

label
original    1658
Fake        1599
Name: count, dtype: int64


In [ ]:
# Save to a new CSV file
train_df.to_csv('/content/Preprocessed_Fake_train.csv', index=False, encoding='utf-8')


In [ ]:
# Split train and dev data
X_train = tokenizer.texts_to_sequences(train_df[text_column])
X_dev = tokenizer.texts_to_sequences(df2[text_column])  # Assuming 'df2' is your dev dataset

# Pad sequences
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post')
X_dev = pad_sequences(X_dev, maxlen=max_sequence_length, padding='post', truncating='post')

# Label encoding for dev set
y_dev = label_encoder.transform(df2[label_column])  # Using same encoder as for the train set


In [ ]:
'''#lstm
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2

# Compute class weights for binary classification
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),  # Use original binary labels (0 and 1)
    y=y_train
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Define the LSTM model with regularization
model = Sequential([
    Embedding(input_dim=max_vocab_size, output_dim=128, input_length=max_sequence_length),  # Embedding layer
    LSTM(64, return_sequences=False),  # LSTM layer
    Dropout(0.5),  # Dropout to avoid overfitting
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # Fully connected layer with L2 regularization
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Binary classification loss
    metrics=['accuracy']
)

# Model summary
model.build(input_shape=(None, max_sequence_length))  # Explicitly build the model with input shape
model.summary()

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_dev, y_dev),
    epochs=50,
    batch_size=16,  # Lower batch size for better optimization in smaller datasets
    class_weight=class_weights_dict,  # Add class weights if class imbalance exists
    verbose=1
)
'''

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,337,857 (5.10 MB)

 Trainable params: 1,337,857 (5.10 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
204/204 ━━━━━━━━━━━━━━━━━━━━ 24s 84ms/step - accuracy: 0.4809 - loss: 1.1413 - val_accuracy: 0.5055 - val_loss: 0.7173
Epoch 2/50
204/204 ━━━━━━━━━━━━━━━━━━━━ 18s 72ms/step - accuracy: 0.5135 - loss: 0.7065 - val_accuracy: 0.4982 - val_loss: 0.6946
Epoch 3/50
204/204 ━━━━━━━━━━━━━━━━━━━━ 20s 71ms/step - accuracy: 0.4936 - loss: 0.6944 - val_accuracy: 0.4982 - val_loss: 0.6934
Epoch 4/50
204/204 ━━━━━━━━━━━━━━━━━━━━ 14s 70ms/step - accuracy: 0.5051 - loss: 0.6932 - val_accuracy: 0.5067 - val_loss: 0.6932
Epoch 5/50
204/204 ━━━━━━━━━━━━━━━━━━━━ 21s 72ms/step - accuracy: 0.5037 - loss: 0.6927 - val_accuracy: 0.5018 - val_loss: 0.6965
Epoch 6/50
204/204 ━━━━━━━━━━━━━━━━━━━━ 20s 72ms/step - accuracy: 0.5120 - loss: 0.6961 - val_accuracy: 0.5018 - val_loss: 0.6942
Epoch 7/50
204/204 ━━━━━━━━━━━━━━━━━━━━ 21s 73ms/step - accuracy: 0.5194 - loss: 0.6939 - val_accuracy: 0.4982 - val_loss: 0.6936
Epoch 8/50
204/204 ━━━━━━━━━━━━━━━━━━━━ 21s 73ms/step - accuracy: 0.4953 - loss: 0.6939 - 

In [ ]:
'''# Evaluate the model on the dev set using lstm
val_loss, val_accuracy = model.evaluate(X_dev, y_dev)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")
'''


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5077 - loss: 0.6931
Validation Loss: 0.6931481957435608, Validation Accuracy: 0.4981594979763031


In [ ]:
'''from sklearn.metrics import classification_report

y_dev_pred = model.predict(X_dev)
y_dev_labels = (y_dev_pred > 0.5).astype(int)
print(classification_report(y_dev, y_dev_labels))
'''

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
              precision    recall  f1-score   support

           0       0.50      1.00      0.67       406
           1       0.00      0.00      0.00       409

    accuracy                           0.50       815
   macro avg       0.25      0.50      0.33       815
weighted avg       0.25      0.50      0.33       815



In [ ]:
'''# Tokenize and pad the test dataset
X_test = tokenizer.texts_to_sequences(test_df[text_column])
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')

# Make predictions
y_test_pred = model.predict(X_test)
y_test_labels = (y_test_pred > 0.5).astype(int)  # Convert probabilities to binary labels

# Save predictions
test_df['label'] = label_encoder.inverse_transform(y_test_labels.flatten())
test_df.to_csv('/content/lstm_Predicted_Fake_test.csv', index=False, encoding='utf-8')
y_test = label_encoder.transform(test_df[label_column])
print("Predictions saved to 'lstm_Predicted_Fake_test.csv'.")
'''

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Predictions saved to 'lstm_Predicted_Fake_test.csv'.


In [ ]:
'''predicted_df = pd.read_csv('/content/lstm_Predicted_Fake_test.csv')
predicted_df
'''

,Id,text,label
0,Fake_01,5000 ഉള്ള പോൾ ലോഗ്‌ഡ്‌വൻ ഇപ്പോള് 250000 എന്താ...,Fake
1,Fake_02,ഓഷോ രജനീഷ് പറഞ്ഞപോലെ എനിക്കപ്പോൾ തോന്നിയത് അ...,Fake
2,Fake_03,ചേട്ടാ വാർത്ത വയ്ക്കുന്നത് കേരളത്തിലാണ് സം...,Fake
3,Fake_04,Shame for entire Woman&#39;s of Kerala,Fake
4,Fake_05,135 code janaghal andhu wide business cheythal...,Fake
...,...,...,...
1014,Fake_1015,Correct ...China cheitha ...weapon spread ing ...,Fake
1015,Fake_1016,ഈ WHO പറയുന്നതനുസരിച്ചു ചികിത്സയും ലോക്ക് ഡൌൺ ...,Fake
1016,Fake_1017,Mask illa aarkum😏😏,Fake
1017,Fake_1018,ഇയാളെ കൊറോണ. രോഗി കൾ കിടയിൽ. ഇടാമായിരുന്നു---!!,Fake


In [ ]:
'''import pandas as pd

# Load the predictions dataset
predictions_data = pd.read_csv('/content/lstm_Predicted_Fake_test.csv')

# Map numeric predictions (0 and 1) to descriptive labels
label_mapping = {0: 'Fake', 1: 'original'}
predictions_data['label'] = predictions_data['label'].map(label_mapping)

# Keep only the 'id' and 'predicted_label' columns
output_data = predictions_data[['Id', 'label']]

# Save the results to a new CSV file
output_path = '/content/lstm_cleaned_test.csv'
output_data.to_csv(output_path, index=False)

print(f"Cleaned predictions saved to {output_path}")
'''


Cleaned predictions saved to /content/lstm_cleaned_test.csv


In [ ]:
#bilstm
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2

# Compute class weights for binary classification
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),  # Use original binary labels (0 and 1)
    y=y_train
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Define the BiLSTM model with regularization
model = Sequential([
    Embedding(input_dim=max_vocab_size, output_dim=128, input_length=max_sequence_length),  # Embedding layer
    Bidirectional(LSTM(64, return_sequences=False)),  # BiLSTM layer
    Dropout(0.8),  # Dropout to avoid overfitting
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # Fully connected layer with L2 regularization
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Binary classification loss
    metrics=['accuracy']
)
# Model summary
model.build(input_shape=(None, max_sequence_length))  # Explicitly build the model with input shape
model.summary()

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_dev, y_dev),
    epochs=10,
    batch_size=16,  # Lower batch size for better optimization in smaller datasets
    class_weight=class_weights_dict,  # Add class weights if class imbalance exists
    verbose=1
)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,395,457 (5.32 MB)

 Trainable params: 1,395,457 (5.32 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 31s 124ms/step - accuracy: 0.5310 - loss: 1.3277 - val_accuracy: 0.6528 - val_loss: 0.6908
Epoch 2/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 40s 119ms/step - accuracy: 0.7433 - loss: 0.5885 - val_accuracy: 0.7890 - val_loss: 0.5185
Epoch 3/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 26s 130ms/step - accuracy: 0.9282 - loss: 0.2522 - val_accuracy: 0.8160 - val_loss: 0.4563
Epoch 4/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 26s 127ms/step - accuracy: 0.9662 - loss: 0.1371 - val_accuracy: 0.8209 - val_loss: 0.4868
Epoch 5/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 42s 130ms/step - accuracy: 0.9719 - loss: 0.1108 - val_accuracy: 0.8233 - val_loss: 0.5570
Epoch 6/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 41s 128ms/step - accuracy: 0.9759 - loss: 0.0875 - val_accuracy: 0.8135 - val_loss: 0.5397
Epoch 7/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 25s 121ms/step - accuracy: 0.9758 - loss: 0.0789 - val_accuracy: 0.8147 - val_loss: 0.6305
Epoch 8/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 42s 126ms/step - accuracy: 0.9851 - loss: 0

In [ ]:
# Evaluate the model on the dev set using bilstm
val_loss, val_accuracy = model.evaluate(X_dev, y_dev)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

from sklearn.metrics import classification_report

y_dev_pred = model.predict(X_dev)
y_dev_labels = (y_dev_pred > 0.5).astype(int)
print(classification_report(y_dev, y_dev_labels))


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.8180 - loss: 0.6872
Validation Loss: 0.6479144096374512, Validation Accuracy: 0.8245398998260498
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step
              precision    recall  f1-score   support

           0       0.85      0.79      0.82       406
           1       0.81      0.86      0.83       409

    accuracy                           0.82       815
   macro avg       0.83      0.82      0.82       815
weighted avg       0.83      0.82      0.82       815



In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the test dataset
test_data = pd.read_csv('/content/Fake_test_without_labels.csv')

# Assuming the text column in the test dataset is named 'text'
test_texts = test_data['text'].values

# Tokenize and pad the test texts (ensure you use the same tokenizer as training)
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

# Predict labels for the test set
test_predictions = model.predict(test_padded)

# Convert predictions to binary labels (threshold = 0.5)
test_labels = (test_predictions > 0.5).astype(int).flatten()

# Add predictions to the test dataframe
test_data['predicted_label'] = test_labels

# Save the results to a new CSV file with the model name in the filename
output_path = '/content/bilstm_predicted_test.csv'
test_data.to_csv(output_path, index=False)

print(f"Predictions saved to {output_path}")


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step
Predictions saved to /content/bilstm_predicted_test.csv


In [ ]:
import pandas as pd

# Load the predictions dataset
predictions_data = pd.read_csv('/content/bilstm_predicted_test.csv')

# Map numeric predictions (0 and 1) to descriptive labels
label_mapping = {0: 'Fake', 1: 'original'}
predictions_data['predicted_label'] = predictions_data['predicted_label'].map(label_mapping)

# Keep only the 'id' and 'predicted_label' columns
output_data = predictions_data[['Id', 'predicted_label']]

# Save the results to a new CSV file
output_path = '/content/bilstm_cleaned_test.csv'
output_data.to_csv(output_path, index=False)

print(f"Cleaned predictions saved to {output_path}")
df=pd.read_csv('/content/bilstm_cleaned_test.csv')
df

Cleaned predictions saved to /content/bilstm_cleaned_test.csv


,Id,predicted_label
0,Fake_01,original
1,Fake_02,Fake
2,Fake_03,original
3,Fake_04,original
4,Fake_05,Fake
...,...,...
1014,Fake_1015,Fake
1015,Fake_1016,Fake
1016,Fake_1017,original
1017,Fake_1018,Fake


In [ ]:
test_df = pd.read_csv('/content/Fake_test_with_labels.csv', encoding='utf-8')
df = pd.read_csv('/content/Fake_test_with_labels.csv')
df

,text,label
0,5000 ഉള്ള പോൾ ലോഗ്‌ഡ്‌വൻ ഇപ്പോള് 250000 എന്താ...,Fake
1,ഓഷോ രജനീഷ് പറഞ്ഞപോലെ എനിക്കപ്പോൾ തോന്നിയത് അ...,Fake
2,ചേട്ടാ വാർത്ത വയ്ക്കുന്നത് കേരളത്തിലാണ് സം...,Fake
3,Shame for entire Woman&#39;s of Kerala,original
4,135 code janaghal andhu wide business cheythal...,Fake
...,...,...
1014,Correct ...China cheitha ...weapon spread ing ...,Fake
1015,ഈ WHO പറയുന്നതനുസരിച്ചു ചികിത്സയും ലോക്ക് ഡൌൺ ...,Fake
1016,Mask illa aarkum😏😏,original
1017,ഇയാളെ കൊറോണ. രോഗി കൾ കിടയിൽ. ഇടാമായിരുന്നു---!!,Fake


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

# Load the files
predicted_file = pd.read_csv('/content/bilstm_cleaned_test.csv')
actual_file = pd.read_csv('/content/Fake_test_with_labels.csv')

# Verify column names
print("Predicted file columns:", predicted_file.columns)
print("Actual file columns:", actual_file.columns)

# Create the 'Id' column in the actual file
# Assuming the row order corresponds to IDs like Fake_01, Fake_02
actual_file['Id'] = ['Fake_{:02d}'.format(i + 1) for i in range(len(actual_file))]

# Verify the 'Id' column
print("Actual file with 'Id' column:")
print(actual_file.head())

# Merge the files on the 'Id' column
merged = pd.merge(predicted_file, actual_file, on='Id')

# Extract the true labels and predicted labels
y_true = merged['label']  # Actual labels
y_pred = merged['predicted_label']  # Predicted labels

# Compare predictions and compute metrics
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Save the merged file with comparison for review (optional)
merged.to_csv('/content/bilstm_merged_comparison.csv', index=False)
print("Merged comparison file saved to /content/bilstm_merged_comparison.csv")


Predicted file columns: Index(['Id', 'predicted_label'], dtype='object')
Actual file columns: Index(['text', 'label'], dtype='object')
Actual file with 'Id' column:
                                                text     label       Id
0  5000 ഉള്ള പോൾ  ലോഗ്‌ഡ്‌വൻ ഇപ്പോള് 250000 എന്താ...      Fake  Fake_01
1  ഓഷോ രജനീഷ്  പറഞ്ഞപോലെ  എനിക്കപ്പോൾ തോന്നിയത് അ...      Fake  Fake_02
2  ചേട്ടാ  വാർത്ത  വയ്ക്കുന്നത്  കേരളത്തിലാണ്  സം...      Fake  Fake_03
3             Shame for entire Woman&#39;s of Kerala  original  Fake_04
4  135 code janaghal andhu wide business cheythal...      Fake  Fake_05
Accuracy: 0.7978410206084396
Classification Report:
              precision    recall  f1-score   support

        Fake       0.81      0.78      0.79       507
    original       0.79      0.82      0.80       512

    accuracy                           0.80      1019
   macro avg       0.80      0.80      0.80      1019
weighted avg       0.80      0.80      0.80      1019

Merged comparison file s

In [ ]:
#lstm+bilstm
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2

# Compute class weights for binary classification
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),  # Use original binary labels (0 and 1)
    y=y_train
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Define the LSTM + BiLSTM model with regularization
model = Sequential([
    Embedding(input_dim=max_vocab_size, output_dim=128, input_length=max_sequence_length),  # Embedding layer
    LSTM(64, return_sequences=True),  # LSTM layer with return_sequences=True for stacking
    Bidirectional(LSTM(64, return_sequences=False)),  # BiLSTM layer
    Dropout(0.8),  # Dropout to avoid overfitting
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # Fully connected layer with L2 regularization
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Binary classification loss
    metrics=['accuracy']
)
# Model summary
model.build(input_shape=(None, max_sequence_length))  # Explicitly build the model with input shape
model.summary()

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_dev, y_dev),
    epochs=10,
    batch_size=16,  # Lower batch size for better optimization in smaller datasets
    class_weight=class_weights_dict,  # Add class weights if class imbalance exists
    verbose=1
)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 100, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100, 64)             │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,412,097 (5.39 MB)

 Trainable params: 1,412,097 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 42s 174ms/step - accuracy: 0.5768 - loss: 1.3404 - val_accuracy: 0.7129 - val_loss: 0.6231
Epoch 2/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 39s 164ms/step - accuracy: 0.8532 - loss: 0.4518 - val_accuracy: 0.7779 - val_loss: 0.4752
Epoch 3/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 41s 164ms/step - accuracy: 0.9202 - loss: 0.2561 - val_accuracy: 0.7951 - val_loss: 0.6814
Epoch 4/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 42s 168ms/step - accuracy: 0.9611 - loss: 0.1384 - val_accuracy: 0.8233 - val_loss: 0.5216
Epoch 5/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 40s 162ms/step - accuracy: 0.9680 - loss: 0.1092 - val_accuracy: 0.8147 - val_loss: 0.6205
Epoch 6/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 42s 165ms/step - accuracy: 0.9771 - loss: 0.0795 - val_accuracy: 0.8196 - val_loss: 0.7397
Epoch 7/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 41s 165ms/step - accuracy: 0.9744 - loss: 0.0789 - val_accuracy: 0.8184 - val_loss: 0.7374
Epoch 8/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 41s 164ms/step - accuracy: 0.9792 - loss: 0

In [ ]:
# Evaluate the model on the dev set
val_loss, val_accuracy = model.evaluate(X_dev, y_dev)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

from sklearn.metrics import classification_report

y_dev_pred = model.predict(X_dev)
y_dev_labels = (y_dev_pred > 0.5).astype(int)
print(classification_report(y_dev, y_dev_labels))


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.7946 - loss: 0.7524
Validation Loss: 0.7586653232574463, Validation Accuracy: 0.7975460290908813
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step
              precision    recall  f1-score   support

           0       0.84      0.74      0.78       406
           1       0.77      0.86      0.81       409

    accuracy                           0.80       815
   macro avg       0.80      0.80      0.80       815
weighted avg       0.80      0.80      0.80       815



In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the test dataset
test_data = pd.read_csv('/content/Fake_test_without_labels.csv')

# Assuming the text column in the test dataset is named 'text'
test_texts = test_data['text'].values

# Tokenize and pad the test texts (ensure you use the same tokenizer as training)
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

# Predict labels for the test set
test_predictions = model.predict(test_padded)

# Convert predictions to binary labels (threshold = 0.5)
test_labels = (test_predictions > 0.5).astype(int).flatten()

# Add predictions to the test dataframe
test_data['predicted_label'] = test_labels

# Save the results to a new CSV file with the model name in the filename
output_path = '/content/lstm+bilstm_predicted_test.csv'
test_data.to_csv(output_path, index=False)

print(f"Predictions saved to {output_path}")


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
Predictions saved to /content/lstm+bilstm_predicted_test.csv


In [ ]:
import pandas as pd

# Load the predictions dataset
predictions_data = pd.read_csv('/content/lstm+bilstm_predicted_test.csv')

# Map numeric predictions (0 and 1) to descriptive labels
label_mapping = {0: 'Fake', 1: 'original'}
predictions_data['predicted_label'] = predictions_data['predicted_label'].map(label_mapping)

# Keep only the 'id' and 'predicted_label' columns
output_data = predictions_data[['Id', 'predicted_label']]

# Save the results to a new CSV file
output_path = '/content/lstm+bilstm_cleaned_test.csv'
output_data.to_csv(output_path, index=False)

print(f"Cleaned predictions saved to {output_path}")



Cleaned predictions saved to /content/lstm+bilstm_cleaned_test.csv


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

# Load the files
predicted_file = pd.read_csv('/content/lstm+bilstm_cleaned_test.csv')
actual_file = pd.read_csv('/content/Fake_test_with_labels.csv')

# Verify column names
print("Predicted file columns:", predicted_file.columns)
print("Actual file columns:", actual_file.columns)

# Create the 'Id' column in the actual file
# Assuming the row order corresponds to IDs like Fake_01, Fake_02
actual_file['Id'] = ['Fake_{:02d}'.format(i + 1) for i in range(len(actual_file))]

# Verify the 'Id' column
print("Actual file with 'Id' column:")
print(actual_file.head())

# Merge the files on the 'Id' column
merged = pd.merge(predicted_file, actual_file, on='Id')

# Extract the true labels and predicted labels
y_true = merged['label']  # Actual labels
y_pred = merged['predicted_label']  # Predicted labels

# Compare predictions and compute metrics
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Save the merged file with comparison for review (optional)
merged.to_csv('/content/lstm+bilstm_merged_comparison.csv', index=False)
print("Merged comparison file saved to /content/lstm+bilstm_merged_comparison.csv")


Predicted file columns: Index(['Id', 'predicted_label'], dtype='object')
Actual file columns: Index(['text', 'label'], dtype='object')
Actual file with 'Id' column:
                                                text     label       Id
0  5000 ഉള്ള പോൾ  ലോഗ്‌ഡ്‌വൻ ഇപ്പോള് 250000 എന്താ...      Fake  Fake_01
1  ഓഷോ രജനീഷ്  പറഞ്ഞപോലെ  എനിക്കപ്പോൾ തോന്നിയത് അ...      Fake  Fake_02
2  ചേട്ടാ  വാർത്ത  വയ്ക്കുന്നത്  കേരളത്തിലാണ്  സം...      Fake  Fake_03
3             Shame for entire Woman&#39;s of Kerala  original  Fake_04
4  135 code janaghal andhu wide business cheythal...      Fake  Fake_05
Accuracy: 0.8007850834151129
Classification Report:
              precision    recall  f1-score   support

        Fake       0.83      0.75      0.79       507
    original       0.77      0.85      0.81       512

    accuracy                           0.80      1019
   macro avg       0.80      0.80      0.80      1019
weighted avg       0.80      0.80      0.80      1019

Merged comparison file s

In [ ]:
#cnn+bilstm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout

# Define the model with CNN + BiLSTM
model = Sequential([
    Embedding(input_dim=max_vocab_size, output_dim=128, input_length=max_sequence_length),  # Embedding layer
    Conv1D(128, 5, activation='relu'),  # CNN layer with 128 filters, kernel size of 5
    MaxPooling1D(pool_size=2),  # MaxPooling to down-sample the sequence
    Bidirectional(LSTM(64, return_sequences=False)),  # BiLSTM layer
    Dropout(0.9),  # Dropout to avoid overfitting
    Dense(64, activation='relu'),  # Fully connected layer
    Dropout(0.8),  # Dropout again to avoid overfitting
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Binary classification loss
    metrics=['accuracy']
)

# Model summary
model.build(input_shape=(None, max_sequence_length))  # Explicitly build the model with input shape
model.summary()

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_dev, y_dev),
    epochs=10,
    batch_size=32,
    verbose=1
)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 100, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 96, 128)             │          82,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 48, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,469,185 (5.60 MB)

 Trainable params: 1,469,185 (5.60 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 20s 142ms/step - accuracy: 0.5152 - loss: 0.6970 - val_accuracy: 0.5914 - val_loss: 0.6895
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 19s 132ms/step - accuracy: 0.5383 - loss: 0.6891 - val_accuracy: 0.6282 - val_loss: 0.6442
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 21s 134ms/step - accuracy: 0.6308 - loss: 0.6387 - val_accuracy: 0.7018 - val_loss: 0.5879
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 14s 134ms/step - accuracy: 0.7921 - loss: 0.4742 - val_accuracy: 0.7546 - val_loss: 0.5075
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 21s 137ms/step - accuracy: 0.8839 - loss: 0.2979 - val_accuracy: 0.8221 - val_loss: 0.4433
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 21s 141ms/step - accuracy: 0.9349 - loss: 0.1702 - val_accuracy: 0.8356 - val_loss: 0.5133
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 14s 134ms/step - accuracy: 0.9588 - loss: 0.1191 - val_accuracy: 0.8196 - val_loss: 0.6941
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 14s 133ms/step - accuracy: 0.9735 - loss: 0

In [ ]:
# Evaluate the model on the dev set
val_loss, val_accuracy = model.evaluate(X_dev, y_dev)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

from sklearn.metrics import classification_report

y_dev_pred = model.predict(X_dev)
y_dev_labels = (y_dev_pred > 0.5).astype(int)
print(classification_report(y_dev, y_dev_labels))


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.8269 - loss: 1.1992
Validation Loss: 1.1909409761428833, Validation Accuracy: 0.8245398998260498
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       406
           1       0.83      0.82      0.82       409

    accuracy                           0.82       815
   macro avg       0.82      0.82      0.82       815
weighted avg       0.82      0.82      0.82       815



In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the test dataset
test_data = pd.read_csv('/content/Fake_test_without_labels.csv')

# Assuming the text column in the test dataset is named 'text'
test_texts = test_data['text'].values

# Tokenize and pad the test texts (ensure you use the same tokenizer as training)
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

# Predict labels for the test set
test_predictions = model.predict(test_padded)

# Convert predictions to binary labels (threshold = 0.5)
test_labels = (test_predictions > 0.5).astype(int).flatten()

# Add predictions to the test dataframe
test_data['predicted_label'] = test_labels

# Save the results to a new CSV file with the model name in the filename
output_path = '/content/cnn+bilstm_predicted_test.csv'
test_data.to_csv(output_path, index=False)

print(f"Predictions saved to {output_path}")


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
Predictions saved to /content/cnn+bilstm_predicted_test.csv


In [ ]:
import pandas as pd

# Load the predictions dataset
predictions_data = pd.read_csv('/content/cnn+bilstm_predicted_test.csv')

# Map numeric predictions (0 and 1) to descriptive labels
label_mapping = {0: 'Fake', 1: 'original'}
predictions_data['predicted_label'] = predictions_data['predicted_label'].map(label_mapping)

# Keep only the 'id' and 'predicted_label' columns
output_data = predictions_data[['Id', 'predicted_label']]

# Save the results to a new CSV file
output_path = '/content/cnn+bilstm_cleaned_test.csv'
output_data.to_csv(output_path, index=False)

print(f"Cleaned predictions saved to {output_path}")



Cleaned predictions saved to /content/cnn+bilstm_cleaned_test.csv


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

# Load the files
predicted_file = pd.read_csv('/content/cnn+bilstm_cleaned_test.csv')
actual_file = pd.read_csv('/content/Fake_test_with_labels.csv')

# Verify column names
print("Predicted file columns:", predicted_file.columns)
print("Actual file columns:", actual_file.columns)

# Create the 'Id' column in the actual file
# Assuming the row order corresponds to IDs like Fake_01, Fake_02
actual_file['Id'] = ['Fake_{:02d}'.format(i + 1) for i in range(len(actual_file))]

# Verify the 'Id' column
print("Actual file with 'Id' column:")
print(actual_file.head())

# Merge the files on the 'Id' column
merged = pd.merge(predicted_file, actual_file, on='Id')

# Extract the true labels and predicted labels
y_true = merged['label']  # Actual labels
y_pred = merged['predicted_label']  # Predicted labels

# Compare predictions and compute metrics
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Save the merged file with comparison for review (optional)
merged.to_csv('/content/cnn+bilstm_merged_comparison.csv', index=False)
print("Merged comparison file saved to /content/cnn+bilstm_merged_comparison.csv")


Predicted file columns: Index(['Id', 'predicted_label'], dtype='object')
Actual file columns: Index(['text', 'label'], dtype='object')
Actual file with 'Id' column:
                                                text     label       Id
0  5000 ഉള്ള പോൾ  ലോഗ്‌ഡ്‌വൻ ഇപ്പോള് 250000 എന്താ...      Fake  Fake_01
1  ഓഷോ രജനീഷ്  പറഞ്ഞപോലെ  എനിക്കപ്പോൾ തോന്നിയത് അ...      Fake  Fake_02
2  ചേട്ടാ  വാർത്ത  വയ്ക്കുന്നത്  കേരളത്തിലാണ്  സം...      Fake  Fake_03
3             Shame for entire Woman&#39;s of Kerala  original  Fake_04
4  135 code janaghal andhu wide business cheythal...      Fake  Fake_05
Accuracy: 0.8066732090284593
Classification Report:
              precision    recall  f1-score   support

        Fake       0.79      0.83      0.81       507
    original       0.82      0.79      0.80       512

    accuracy                           0.81      1019
   macro avg       0.81      0.81      0.81      1019
weighted avg       0.81      0.81      0.81      1019

Merged comparison file s